In [2]:
import os
import dlib
import glob

import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

import numpy as np
%matplotlib inline

In [3]:
img_black = np.zeros((70,140,3))

base_dir = 'data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

classes = 'Begin, Choose, Connection, Navigation, Next, Previous, Start, Stop, Hello, Web'
classes = classes.split(', ')

predictor_path = 'assests/predictors/shape_predictor_68_face_landmarks.dat'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

train_people = 'F01, F02, F04, F05, F06, F07, F08, F09, M01, M02, M04'.split(', ')
val_people = 'F10, M07'.split(', ')
test_people = 'F11, M08'.split(', ')
print(train_people)
print(val_people)
print(test_people)

classes_num = ['0'+str(i) if i < 10 else str(i) for i in range(1, 11) ]
word_ids = ['0'+str(i) if i < 10 else str(i) for i in range(1, 11) ]
classes_dict = dict(zip(classes_num, classes))
print(classes_num)
print(word_ids)
print(classes_dict)

['F01', 'F02', 'F04', 'F05', 'F06', 'F07', 'F08', 'F09', 'M01', 'M02', 'M04']
['F10', 'M07']
['F11', 'M08']
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']
{'01': 'Begin', '02': 'Choose', '03': 'Connection', '04': 'Navigation', '05': 'Next', '06': 'Previous', '07': 'Start', '08': 'Stop', '09': 'Hello', '10': 'Web'}


In [4]:
def show_img(img, figsize=(8, 8)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.grid(False)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.imshow(img)
    plt.imshow(img)

In [5]:
train_image = []
train_class = []

for class_id in range(len(classes)):
    images = sorted(glob.glob('data/train/' + classes[class_id] + '/*.jpg'))
    for i in range(len(images)):
        if ('noised' in images[i] or 'rand_contr' in images[i]
        or 'vert_flip' in images[i] or 'hor_flip' in images[i]):
            train_image.append(images[i].split('/')[3])
            train_class.append(classes_dict[images[i].split('/')[3].split(']')[-1][:2]])
        else:
            train_image.append(images[i].split('/')[3])
            train_class.append(classes_dict[images[i].split('/')[3].split('_')[0]])
        
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

train_data.to_csv('data/miracl/train_new.csv',header=True, index=False)


val_image = []
val_class = []

for class_id in range(len(classes)):
    images = sorted(glob.glob('data/validation/' + classes[class_id] + '/*.jpg'))
    for i in range(len(images)):
        val_image.append(images[i].split('/')[3])
        val_class.append(classes_dict[images[i].split('/')[3].split('_')[0]])
val_data = pd.DataFrame()
val_data['image'] = val_image
val_data['class'] = val_class

val_data.to_csv('data/miracl/val_new.csv',header=True, index=False)

In [6]:
train = pd.read_csv('data/miracl/train_new.csv')
val = pd.read_csv('data/miracl/val_new.csv')

y_tr = train['class']
y_tr_dummy = pd.get_dummies(y_tr)

y_val = val['class']
y_val_dummy = pd.get_dummies(y_val)

In [7]:
train.head()

,image,class
0,01_F01_01_color_001.jpg,Begin
1,01_F01_01_color_002.jpg,Begin
2,01_F01_01_color_003.jpg,Begin
3,01_F01_01_color_004.jpg,Begin
4,01_F01_01_color_005.jpg,Begin


In [8]:
train.tail()

,image,class
6915,10_M08_10_color_006.jpg,Web
6916,10_M08_10_color_007.jpg,Web
6917,10_M08_10_color_008.jpg,Web
6918,10_M08_10_color_009.jpg,Web
6919,10_M08_10_color_010.jpg,Web


In [9]:
val.head()

,image,class
0,01_F10_01_color_001.jpg,Begin
1,01_F10_01_color_002.jpg,Begin
2,01_F10_01_color_003.jpg,Begin
3,01_F10_01_color_004.jpg,Begin
4,01_F10_01_color_005.jpg,Begin


In [10]:
val.tail()

,image,class
1505,10_M07_10_color_006.jpg,Web
1506,10_M07_10_color_007.jpg,Web
1507,10_M07_10_color_008.jpg,Web
1508,10_M07_10_color_009.jpg,Web
1509,10_M07_10_color_010.jpg,Web


In [11]:
print(len(train))
print(len(val))

6920
1510


In [1]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [2]:
import keras
#from tensorflow import keras

print(tf.__version__)
print(keras.__version__)

2.1.0
2.3.1
Using TensorFlow backend.


In [3]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
from video_datagen import ImageDataGenerator

In [5]:
datagen = ImageDataGenerator()
train_data = datagen.flow_from_directory('data/train', target_size=(70, 140), batch_size=1, frames_per_step=10, shuffle=False)
val_data = datagen.flow_from_directory('data/validation', target_size=(70, 140), batch_size=1, frames_per_step=10, shuffle=False)

Found 6920 images belonging to 10 classes.
Found 1510 images belonging to 10 classes.


In [6]:
import math

print(math.ceil(6920/(1*10)))
print(math.ceil((1510/(1*10))))

692
151


In [7]:
from keras.layers import *
from keras.regularizers import *
from keras.constraints import *

In [8]:
model_small = keras.models.Sequential()

model_small.add(ZeroPadding3D(padding=(1, 2, 2), input_shape=(10, 70, 140, 3)))
model_small.add(Conv3D(32, (3, 5, 5), strides=(1, 2, 2), activation='relu'))
model_small.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model_small.add(Dropout(.2))

model_small.add(ZeroPadding3D(padding=(1, 2, 2)))
model_small.add(Conv3D(64, (3, 5, 5), strides=(1, 1, 1), activation='relu'))
model_small.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model_small.add(Dropout(.2))

model_small.add(ZeroPadding3D(padding=(1, 1, 1)))
model_small.add(Conv3D(96, (3, 3, 3), strides=(1, 2, 2), activation='relu'))
model_small.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model_small.add(Dropout(.2))

model_small.add(TimeDistributed(Flatten()))

# model_small.add(Bidirectional(GRU(10, return_sequences=True)))
model_small.add(Bidirectional(GRU(10)))

model_small.add(Dense(10, activation='softmax'))

In [9]:
adam2 = keras.optimizers.Adam(lr=0.0001)
save_weights = keras.callbacks.ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')
model_small.compile(loss='categorical_crossentropy',optimizer=adam2,metrics=['accuracy'])
model_small.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding3d_1 (ZeroPaddin (None, 12, 74, 144, 3)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 10, 35, 70, 32)    7232      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 10, 17, 35, 32)    0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 17, 35, 32)    0         
_________________________________________________________________
zero_padding3d_2 (ZeroPaddin (None, 12, 21, 39, 32)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 10, 17, 35, 64)    153664    
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 10, 8, 17, 64)    

In [10]:
# history = model_small.fit_generator(train_data, epochs=20, steps_per_epoch=231, 
#                               validation_data=val_data, validation_steps=51, 
#                               callbacks=[save_weights], shuffle=False)

In [11]:
history = model_small.fit_generator(train_data, steps_per_epoch=692, epochs=20, validation_data=val_data, validation_steps=151, shuffle=False)

Epoch 1/20
692/692 [==============================] - 55s 79ms/step - loss: 2.6256 - accuracy: 0.0896 - val_loss: 2.7680 - val_accuracy: 0.0596
Epoch 2/20
692/692 [==============================] - 49s 71ms/step - loss: 2.5071 - accuracy: 0.0997 - val_loss: 2.5843 - val_accuracy: 0.0464
Epoch 3/20
692/692 [==============================] - 47s 68ms/step - loss: 2.4258 - accuracy: 0.0939 - val_loss: 2.3817 - val_accuracy: 0.0464
Epoch 4/20
692/692 [==============================] - 44s 63ms/step - loss: 2.4016 - accuracy: 0.1503 - val_loss: 1.3630 - val_accuracy: 0.0861
Epoch 5/20
692/692 [==============================] - 45s 65ms/step - loss: 2.4351 - accuracy: 0.0838 - val_loss: 1.8117 - val_accuracy: 0.0464
Epoch 6/20
692/692 [==============================] - 44s 64ms/step - loss: 2.4044 - accuracy: 0.1156 - val_loss: 1.5778 - val_accuracy: 0.0728
Epoch 7/20
692/692 [==============================] - 44s 64ms/step - loss: 2.4317 - accuracy: 0.0882 - val_loss: 2.4108 - val_accuracy:

In [ ]:
# video = keras.layers.Input(shape=(10, 70, 140, 3))

# cnn_base = keras.applications.MobileNetV2(input_shape=(70, 140, 3), include_top=False)
# cnn_out = keras.layers.GlobalAveragePooling2D()(cnn_base.output)

# cnn = Model(input=cnn_base.input, output=cnn_out)
# encoded_frames = keras.layers.TimeDistributed(cnn)(video)

# encoded_sequence = keras.layers.Bidirectional(LSTM(10))(encoded_frames)
# outputs = keras.layers.Bidirectional(Dense(10, activation="softmax"))(encoded_sequence)

# mn_lstm = Model([video], outputs)

In [ ]:
# adam2 = keras.optimizers.Adam(lr=0.0001)
# save_weights = keras.callbacks.ModelCheckpoint('mn_lstm.hdf5', save_best_only=True, monitor='val_accuracy', mode='max')
# mn_lstm.compile(loss='categorical_crossentropy',optimizer=adam2,metrics=['accuracy'])
# mn_lstm.summary()

In [ ]:
# history = mn_lstm.fit_generator(train_data, epochs=20, steps_per_epoch=692, 
#                               validation_data=val_data, validation_steps=151, shuffle=False)